In [ ]:
import numpy as np
import torch

In [ ]:
def matrix_nms(boxes, scores, method='gaussian', gaussian_sigma=0.5, post_threshold=0.3):
    """
    Matrix NMS 的 PyTorch 实现
    Args:
        boxes (torch.Tensor): 边界框 [N, 4]
        scores (torch.Tensor): 置信度分数 [N]
        method (str): 衰减方法，可选 'gaussian' 或 'linear'
        gaussian_sigma (float): gaussian 方法的 sigma 参数
        post_threshold (float): 后处理阈值
    Returns:
        torch.Tensor: 更新后的分数
    """
    N = boxes.shape[0]
    if N == 0:
        return torch.zeros(N)

    # 计算边界框的面积
    x1, y1, x2, y2 = boxes[:, 0], boxes[:, 1], boxes[:, 2], boxes[:, 3]
    areas = (x2 - x1) * (y2 - y1)

    # 扩展维度以便后续广播计算
    boxes1 = boxes.unsqueeze(1).expand(N, N, 4)
    boxes2 = boxes.unsqueeze(0).expand(N, N, 4)
    
    # 计算IoU矩阵
    xy_max = torch.min(boxes1[..., 2:], boxes2[..., 2:])
    xy_min = torch.max(boxes1[..., :2], boxes2[..., :2])
    inter = torch.clamp((xy_max - xy_min).prod(dim=2), min=0)
    union = areas.unsqueeze(0) + areas.unsqueeze(1) - inter
    iou = inter / union

    # 获取每个框的最高分数
    max_scores = scores.max()
    
    # 按分数降序排序
    sorted_idx = torch.argsort(scores, descending=True)
    sorted_boxes = boxes[sorted_idx]
    sorted_scores = scores[sorted_idx]
    
    iou = iou[sorted_idx][:, sorted_idx]
    
    # 计算衰减系数
    if method == 'gaussian':
        decay = torch.exp(-iou ** 2 / gaussian_sigma)
    else:  # linear
        decay = (1 - iou) / (1 - iou.min())
    
    # 对角线设为1，避免自身影响
    decay = decay - torch.diag(torch.diag(decay))
    decay = decay + torch.eye(N)

    # 计算衰减后的分数
    decay_scores = sorted_scores * torch.min(decay, dim=0)[0]
    
    # 应用后处理阈值
    keep = decay_scores > post_threshold
    
    return decay_scores, keep

In [ ]:
if __name__ == '__main__':
    # 创建测试数据
    boxes = torch.tensor([
        [100, 100, 200, 200],
        [110, 110, 210, 210],
        [120, 120, 220, 220],
        [130, 130, 230, 230],
        [200, 200, 300, 300],
    ], dtype=torch.float32)
    
    scores = torch.tensor([0.9, 0.8, 0.7, 0.6, 0.95], dtype=torch.float32)
    
    print("原始检测框：")
    for i, (box, score) in enumerate(zip(boxes, scores)):
        print(f"Box {i+1}: {box.tolist()}, score: {score:.3f}")
    
    # 运行Matrix NMS
    decay_scores, keep = matrix_nms(
        boxes,
        scores,
        method='gaussian',
        gaussian_sigma=0.5,
        post_threshold=0.3
    )
    
    print("\nMatrix NMS 后的结果：")
    for i, (box, old_score, new_score, is_keep) in enumerate(zip(boxes, scores, decay_scores, keep)):
        if is_keep:
            print(f"Box {i+1}: {box.tolist()}")
            print(f"Original score: {old_score:.3f}, Decayed score: {new_score:.3f}")
    
    print("\n统计信息：")
    print(f"原始检测框数量: {len(boxes)}")
    print(f"保留检测框数量: {keep.sum().item()}")
    print(f"检测框保留率: {keep.sum().item() / len(boxes) * 100:.1f}%")